# WebScraping of Car Data

In [1]:
# Importing Libraries

import selenium
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import requests
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from collections import Counter

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

#Importing Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
def cars24_search(search_term,location):
    """Generate a URL for search term with location"""
    
    LOC = {'Delhi': '110001', 'Noida': '201301', 'Gurgoan': '122001', 'Mumbai' : '400001', 'Pune': '411001', 
           'Banglore': '560001','Hyderabad': '500001', 'Chennai': '600001', 'Kolkata': '700001', 'Ahmedabad': '380001' }
    
    search_term = search_term.replace(' ','%20')
    template = f'https://www.cars24.com/buy-used-car?sort=P&search={search_term}&storeCityId=8597&pinId={LOC[location]}'
    
    return template

In [3]:
def cars24_scrapping():
    """ 
    Function to scrap datas from cars 24.
    
    """
      
    print("SCRAPPING THE REQUIRED DETAILS")
    driver = webdriver.Chrome(r"D:\Downloads\chromedriver_win\chromedriver.exe")
    driver.maximize_window()
    
    LOC = {'Delhi': '110001', 'Noida': '201301', 'Gurgoan': '122001', 'Mumbai' : '400001', 'Pune': '411001', 
           'Banglore': '560001','Hyderabad': '500001', 'Chennai': '600001', 'Kolkata': '700001', 'Ahmedabad': '380001' }
    
    location = [keys for keys in LOC.keys()]
    
    CARS= ['SUV', 'Sedan', 'Hatchback', 'Luxury Sedan', 'Luxury SUV']
    
    URL = [cars24_search(j,i) for i in location for j in CARS]
    
    LOCATION = []
    MNF_YEAR = []
    BRAND = []
    MODEL = []
    VARIANT = []
    driven_KM = []
    NOOF_owners = []
    fuel_type = []
    PRIZE = []


    for i in URL:
        driver.get(i)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        time.sleep(10)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'class' : '_1l4fi'})

        description = []
        description1 = []
        description2 = []
        price = []
    
        for i in results:
            description.append(i.find('h2', {'class': "_3FpCg"}).text)
            description1.append(i.find('p', {'class': "cvakB"}).text)
            description2.append(i.find('ul', {'class': "bVR0c"},'li').text)
            price.append(i.find('div', {'class': "_7udZZ"}).text)
        
    
    
        year = [i.split(' ',2)[0] for i in description]
        brand = [i.split(' ',2)[1] for i in description]
        D1= [i.split(' ',2)[-1] for i in description]

        for i in price:
            PRIZE.append(i)

        for i in year:
            MNF_YEAR.append(i)
        for i in brand:
            BRAND.append(i)
    
        D2 = [i.rsplit(' ',1)[0] for i in description1]
        variant = [i.split(' ') [-1]for i in description1]
        for i in variant:
            VARIANT.append(i)
    
        D3 = [D1[i]+' '+D2[i] for i in range(0,len(D1))]
    
        for i in D3:
            MODEL.append(i)
    
        dummy1 = [i.split('km')[-1]for i in description2]
        kms = [i.split('km')[0] for i in description2]
        kms = [i+'km' for i in kms]

        owners = [i.split('Owner')[0] for i in dummy1]
        owners = [i+'Owner' for i in owners]
        fuel = [i.split('Owner')[-1] for i in dummy1]

        for i in kms:
            driven_KM.append(i)

        for i in owners:
            NOOF_owners.append(i)

        for i in fuel:
            fuel_type.append(i)

    driver.close()
        
    used_cars = pd.DataFrame({"MNF_YEAR" : MNF_YEAR,
                              "BRAND" : BRAND,
                              "MODEL" : MODEL,
                              "VARIANT" : VARIANT,
                              "DRIVEN_KM" : driven_KM,
                              "FUELTYPE" : fuel_type,
                              "NOOF_OWNERS" : NOOF_owners,
                              "PRICE" : PRIZE
                             })
    
    used_cars.to_csv("used_cars.csv",index=False) #Creating CSV
    print("SCRAPPED DETAILS ARE STORED AS 'used_cars.csv'")
    
    return used_cars

In [4]:
user_cars = cars24_scrapping()

SCRAPPING THE REQUIRED DETAILS
SCRAPPED DETAILS ARE STORED AS 'used_cars.csv'


In [5]:
user_cars

,MNF_YEAR,BRAND,MODEL,VARIANT,DRIVEN_KM,FUELTYPE,NOOF_OWNERS,PRICE
0,2021,Mahindra,XUV 3OO 1.2 W6 MT,,"8,387 km",PetrolUP-14,1st Owner,"₹9,21,699"
1,2020,Mahindra,XUV 3OO 1.2 W6 MT,,"18,028 km",PetrolDL-10,1st Owner,"₹8,92,649"
2,2022,Mahindra,XUV 3OO 1.2 W6 AT,,"21,859 km",PetrolDL-1C,1st Owner,"₹10,50,499"
3,2018,Mahindra,KUV 100 NXT K2 D 6S,Manual,"65,179 km",DieselDL-10,1st Owner,"₹4,54,599"
4,2020,Mahindra,KUV 100 NXT K8 6 STR,Manual,"26,577 km",PetrolDL-8C,1st Owner,"₹6,89,899"
...,...,...,...,...,...,...,...,...
2015,2019,MG,HECTOR SHARP 2.0 DIESEL,Manual,"55,691 km",DieselGJ-03,2nd Owner,"₹14,95,099"
2016,2020,Honda,WR-V 1.2 i-VTEC VX MT,Manual,"47,489 km",PetrolGJ-06,1st Owner,"₹8,20,599"
2017,2022,Hyundai,Creta SX PETROL MT,Manual,"1,354 km",PetrolGJ-01,1st Owner,"₹15,24,999"
2018,2020,MG,HECTOR SHARP DCT PETROL,Automatic,"38,544 km",PetrolGJ-03,1st Owner,"₹15,59,399"


In [8]:
# Saving the scraped data into excel file
user_cars.to_csv("UsedCars.csv")